# Group 21 Project Report

## Introduction

Heart disease is one of the leading causes of death in Canada. It encompasses all kinds of heart conditions such as coronary heart disease, heart defects, and arrythmia’s (irregularity in heartbeat). To better understand heart disease, it is important to look at the key factors that influence an individual’s risk of having this condition. In this project we will be looking at five different factors that have a correlation with heart disease. These factors are cholesterol, maximum heart rate, sex, age, and chest pain. These will be used as predictors in this project.

So how are each of these  predictors associated with heart disease? Based on scientific literature, high blood pressure places extra exertion on blood vessels, increasing tension on the arteries, which decreases blood flow to major organs and oxygen to the heart. Aside from this, people who have high levels of cholesterol could have fat deposits build up in the arteries, making it  challenging for blood to flow freely through the body. Additionally, regarding maximum heart rate, reaching the ideal heart rate range can indicate a healthy heart in good condition. If it exceeds a healthy range, the maximum heart rate of a patient can be used as an indicator of a potential heart issue. A common factor in many kinds of diseases is an increase in age, which is often correlated with a higher risk for heart disease. Lastly, incidence of chest pain can be a risk factor for heart disease if evaluated at abnormal levels. The data base classifies the chest pain type by typical chest pain (angina), atypical angina, no chest pain, and asymptomatic.

The question we want to answer is: are we able to predict whether a patient has heart disease based on their health profile? 

The information used to answer this question will be from the heart disease dataset, and used to create an accurate prediction model to classify new data. The heart disease data set encompasses 14 factors, in which five of them will be used as predictors and diagnosis of heart disease is what we will be predicting.

## Methods

To answer our research question, we used the heart disease data collected from the Cleveland hospital. Initially, cholesterol levels (chol), resting blood pressure (trestbps), age (age), chest pain (cp), and maximum heart rate (thalach) are the five variables that were used as the predictors in our classification model, due to their aforementioned strong association with heart disease. We used the K-nearest neighbor classification algorithm to predict the presence of heart disease, denoted the “predicted attribute” in our code, which we have simplified to a binary factor that indicates its presence or absence, denoted as "Positive" or "Negative" respectively. For our preliminary exploratory data analysis, we summarized the Cleaveland hospital data into a data frame that grouped the patient data by their respective sex (male or female) and heart condition (Positive or Negative for heart disease) in order to calculate the mean values of three quantifiable predictors: cholesterol, resting blood pressure, and maximum heart rate. This was done to see if patients with heart disease tend to have higher mean values of the aforementioned variables, before visualizating the data. These mean values revelead that a higher maximum heart rate does not necessarily lead to heart disease, as the maximum heart rate achieved in positive patients was actually lower than the mean value in negative patients, which is contrary to our hypothesis. To visualize this trend, a histogram that expressed the distribution of patients with varying maximum heart rates (thalach) was created, which reinforced the notion that a higher maximum heart rate was not measured in patients with heart disease, as positive heart disease cases had a maximum heart rate distribution that was skewed lower compared to negative cases (reaffirmed by a lower median). Likewise, analyzing the mean and median values of cholesterol levels revealed that positive cases had slightly higher distributions of cholesterol levels. To visualize and analyze chest pain as a predictor, we created a bar graph quantifying the number of patients with varying levels of chest pain, 1 corresponding to typical, 2 to atypical, 3 to no pain, and 4 to asymptomatic. Profoundly, this showed that asymptomatic chest pain was associated with positive cases, while the remaining levels of chest pain were mostly found in negative patients. However, grouping by sex revealed that there were differences between males and females in their resting blood pressure, cholesterol levels, and maximum heart rate, suggesting that it may a valuable predictor to add to our model. Regarding the resting blood pressure (trestbps), our summary data of the mean and median blood pressure between positive and negative cases revealed that blood pressure did not appreciably affect the incidence of heart disease, since the medians and means were almost indentical, and were centered similarily on our blood pressure histogram. As such, contrary to our original plan, we believe that blood pressure would not be a useful predictor in our model, and was removed before proceeding further. Altogether, exploratory data analysis resulted in the inclusion of sex into our model, replacing  resting blood pressure, leaving our five predictors as age, sex, chest pain, maximum heart rate, and cholesterol.

75% of the total data will be used as training data, while the remaining 25% will be used as testing data. Five fold cross-validation was used during data training to increase the accuracy of our prediction by determining the optimal K value to tune our model. A scatter plot will be used to visualize the correlation between what we believe to be two major variables and indicators of heart disease, cholesterol and resting blood pressure. The points will be coloured based on the presence of heart disease in each patient. Resting blood pressure and cholesterol will be plotted on the x and y axes respectively. 

## References

Aha, D. (2019). Heart Disease Data Set. UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/citation_policy.html 

Lennep, J., Westerveld, H., Erkelens, D., & Van der Wall, E. (2002). Risk factors for coronary heart disease: implications of gender. Cardiovascular Research. 53(3). 538-549.  https://doi.org/10.1016/S0008-6363(01)00388-1

Thayer, J., Yamamoto, S., & Brosschot, J. (2010). The relationship of autonomic imbalance, heart rate variability and cardiovascular disease risk factors. Internal Journal of Cardiology, 141(2). 122-131. https://doi.org/10.1016/j.ijcard.2009.09.543